In [5]:
import pandas as pd
from zipfile import ZipFile

In [35]:
year = 2021

In [47]:
df = pd.read_csv(ZipFile("data/Germany/2021.zip").open("btw21_wbz_ergebnisse.csv"), delimiter=";")
df = df[~df["Land"].isna()]
df["polling_id"] = df["Wahlkreis"].astype(int).astype(str) + df["Land"].astype(int).astype(str) \
    + df["Gemeinde"].astype(int).astype(str) + df["Wahlbezirk"].astype(str) \
+ df["Bezirksart"].astype(int).astype(str)

In [10]:
list(df)

['Wahlkreis',
 'Land',
 'Regierungsbezirk',
 'Kreis',
 'Verbandsgemeinde',
 'Gemeinde',
 'Kennziffer Urnenwahlbezirke nach § 68 BWO',
 'Kennziffer Briefwahlzugehörigkeit',
 'Gemeinde Name',
 'Wahlbezirk',
 'Bezirksart',
 'Wahlberechtigte (A)',
 'Wahlberechtigte ohne Sperrvermerk (A1)',
 'Wahlberechtigte mit Sperrvermerk (A2)',
 'Wahlberechtigte nach § 25 Abs. 2 BWO (A3)',
 'Wählende (B)',
 'Wählende mit Wahlschein (B1)',
 'Ungültige',
 'Gültige',
 'CDU',
 'SPD',
 'AfD',
 'FDP',
 'DIE LINKE',
 'GRÜNE',
 'CSU',
 'FREIE WÄHLER',
 'Die PARTEI',
 'Tierschutzpartei',
 'NPD',
 'PIRATEN',
 'ÖDP',
 'V-Partei³',
 'DiB',
 'BP',
 'Tierschutzallianz',
 'MLPD',
 'Gesundheitsforschung',
 'MENSCHLICHE WELT',
 'DKP',
 'Die Grauen',
 'BüSo',
 'Die Humanisten',
 'Gartenpartei',
 'du.',
 'dieBasis',
 'Bündnis C',
 'BÜRGERBEWEGUNG',
 'III. Weg',
 'Bündnis21',
 'LIEBE',
 'LKR',
 'SSW',
 'Team Todenhöfer',
 'UNABHÄNGIGE',
 'Volt',
 'Volksabstimmung',
 'B*',
 'sonstige',
 'FAMILIE',
 'Graue Panther',
 'Klimal

In [23]:
party = ["CDU","SPD","AfD","FDP","DIE LINKE","GRÜNE","CSU","FREIE WÄHLER","Die PARTEI","Tierschutzpartei","NPD","PIRATEN","ÖDP","V-Partei³","DiB","BP","Tierschutzallianz","MLPD","Gesundheitsforschung","MENSCHLICHE WELT","DKP","Die Grauen","BüSo","Die Humanisten","Gartenpartei","du.","dieBasis","Bündnis C","BÜRGERBEWEGUNG","III. Weg","Bündnis21","LIEBE","LKR","SSW","Team Todenhöfer","UNABHÄNGIGE","Volt","Volksabstimmung","B*","sonstige","FAMILIE","Graue Panther","KlimalisteBW","THP"]

In [48]:
df_filtered = df[["polling_id"] + party].melt(id_vars=["polling_id"], var_name="candidate")
df_filtered["value"] = df_filtered["value"].fillna(0).astype(int)
df_filtered.head()

,polling_id,candidate,value
0,110010,CDU,129
1,110020,CDU,114
2,110030,CDU,76
3,110040,CDU,54
4,110050,CDU,27


In [49]:
tt = df_filtered.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "rate"]]

In [50]:
df_filtered = pd.merge(df_filtered, tt, on=["polling_id", "candidate"])

In [51]:
df_filtered.to_csv(f"data_output/Germany/{year}.csv.gz", compression="gzip", index=False)

In [40]:
df.Land.unique()

array([ 1., 13.,  2.,  3.,  4., 12., 15., 11.,  5., 14.,  6., 16.,  7.,
        9.,  8., 10.])

In [52]:
df = df.rename(columns={
    "Wahlkreis": "constituency",
    "Land": "state_id",
    "Gemeinde Name": "municipality",
    "Gemeinde": "municipality_id"
})
df[["constituency", "state_id"]] = df[["constituency", "state_id"]].astype(int)
df_location = df[["polling_id", "constituency", "state_id"]].to_csv(f"data_output/Germany/{year}_location.csv.gz", compression="gzip", index=False)

In [ ]:
df_location = df[["polling_id", "state_id", "province_id", "commune_id", "district_commune_id"]].to_csv(f"data_output/Spain/{year}_{month}_location.csv.gz", compression="gzip", index=False)